In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t

import ft_digital_trade.src.utils.read_data as read_utils
import ft_digital_trade.src.utils.clean_utils as clean_utils
import ft_digital_trade.src.utils.calculation_utils as calc_utils
import ft_digital_trade.src.utils.plot_utils as plot_utils

client = bigquery.Client()

In [ ]:
# Getting cardholder totals to scale spend values up (not necessary for this as using %, but useful for some other analysis)
# Calculating Visa marketshare drop-off using change in cardholders over time
# Looks at how total number of UK cardholders in the dataset changes over time to scale each category of spend

cardholders = '''SELECT time_period_value, sum(cardholders) as total_cardholders
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All'
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  GROUP BY time_period_value
  ORDER BY time_period_value ASC
'''
cardholders_total = bq.read_bq_table_sql(client, cardholders)
#cardholders_total

base_cardholders = cardholders_total['total_cardholders'].iloc[0]
#base_cardholder

cardholders_total['Change from Base'] = (base_cardholders / cardholders_total['total_cardholders'])
cardholders_total # Change from Base column can now be multiplied against each quarter's spend values to adjust the spend for Visa's marketshare

In [ ]:
# Total spend in foreign countries by UK cardholders
total_spend_abroad = '''SELECT time_period_value, sum(spend) as total_spend_abroad
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter'
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
spend_abroad = bq.read_bq_table_sql(client, total_spend_abroad)
#spend_abroad.head()

In [ ]:
# Total online spend in foreign countries by UK cardholders
total_online_spend_abroad = '''SELECT time_period_value, sum(spend) as total_online_spend_abroad
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All'
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
online_spend_abroad = bq.read_bq_table_sql(client, total_online_spend_abroad)
online_spend_abroad.head()

In [ ]:
# Total spend in the UK by UK cardholders
total_spend_domestic = '''SELECT time_period_value, sum(spend) as total_spend_domestic
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All' 
  AND destination_country = 'UNITED KINGDOM'
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
spend_domestic = bq.read_bq_table_sql(client, total_spend_domestic)
#spend_domestic.head()

In [ ]:
# Total online spend in UK by UK cardholders
total_online_spend_domestic = '''SELECT time_period_value, sum(spend) as total_online_spend_domestic
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country = 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
online_spend_domestic = bq.read_bq_table_sql(client, total_online_spend_domestic)
#online_spend_domestic.head()

In [ ]:
# Adjusting all 4 series
adjusted_abroad = spend_abroad

In [ ]:
# Merging abroad figures together 
merged_abroad_spending = pd.merge(spend_abroad, online_spend_abroad, on='time_period_value')

# Merging domestic figures together
merged_dom_spending = pd.merge(spend_domestic, online_spend_domestic, on='time_period_value')

#merged_dom_spending

In [ ]:
# Calculating the online % ratio for each

merged_abroad_spending["Abroad Online %"] = (merged_abroad_spending['total_online_spend_abroad'] /  merged_abroad_spending['total_spend_abroad']) * 100
merged_dom_spending["Domestic Online %"] = (merged_dom_spending['total_online_spend_domestic'] /  merged_dom_spending['total_spend_domestic']) * 100

merged_dom_spending